# demo05_htl.ipynb    酒店评论情感分析

In [15]:
import numpy as np
import pandas as pd
import sklearn.model_selection as ms
import sklearn.naive_bayes as nb
import sklearn.metrics as sm
import jieba
jieba.load_userdict('mydict.txt')

In [16]:
# 加载文件
data = pd.read_csv('htl_all.csv')
data = data.tail((data['label']==0).sum() * 2)
data['label'].value_counts()

1    2443
0    2443
Name: label, dtype: int64

In [17]:
def func(item):
    # 分词
    seg_list = jieba.cut(item)
    return ' '.join(seg_list)
    
data['new_review'] = data['review'].apply(func)

In [18]:
import sklearn.feature_extraction.text as ft
# 整理输入集与输出集    TFIDF
cv = ft.CountVectorizer()
bow = cv.fit_transform(data['new_review'])
tt = ft.TfidfTransformer()
tfidf = tt.fit_transform(bow)
# 拆分测试集与训练集
train_x, test_x, train_y, test_y = \
    ms.train_test_split(tfidf, data['label'], test_size=0.1, random_state=7)
# 交叉验证
# 使用朴素贝叶斯
import sklearn.naive_bayes as nb
model = nb.MultinomialNB()
scores = ms.cross_val_score(model, tfidf, data['label'], cv=5, scoring='f1_weighted')
print(scores.mean())

# 训练模型
model.fit(train_x, train_y)
# 测试模型，评估模型
pred_test_y = model.predict(test_x)
print(sm.classification_report(test_y, pred_test_y))

0.8591307667355867
             precision    recall  f1-score   support

          0       0.88      0.88      0.88       252
          1       0.87      0.87      0.87       237

avg / total       0.88      0.88      0.88       489



In [24]:
test_data = ['这家酒店棒，环境棒，电视棒，空调不错，挺好的。',
             '地板脏，空调漏水，再也不来了！',
             '空调不太凉，早餐还行，但是服务差了点，总体一般。',
             '我不喜欢。', "哪哪都好！房价大，视野开阔，环境非常好，下次还来！"]
test_data = pd.Series(test_data).apply(func)
# 把样本按照训练时的方式转换成tfidf矩阵，才可以交个模型进行预测
bow = cv.transform(test_data)
test_data = tt.transform(bow)
pred_test_y = model.predict(test_data)

# 输出置信概率
pred_prob_y = model.predict_proba(test_data)

print(pred_test_y)
probs = pred_prob_y.max(axis=1)
for b, c in zip(pred_test_y, probs):
    print(b, ':', c)

[1 0 1 0 1]
1 : 0.5594777973826913
0 : 0.8753288789559337
1 : 0.6186367347028627
0 : 0.542983041059403
1 : 0.8244575638242924
